In [1]:
import pandas as pd
import numpy as np
import nltk
from google.colab import drive

In [2]:
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
nltk.download("punkt")
nltk.download("punkt_tab")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
file_path = "/content/drive/MyDrive/Datasets/100_Unique_QA_Dataset.csv"
df = pd.read_csv(file_path)
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [5]:
from nltk.tokenize import word_tokenize
def tokenize(text):
    text = text.lower()
    tokens = word_tokenize(text)
    return tokens

In [6]:
tokenize("What is the capital of France?")

['what', 'is', 'the', 'capital', 'of', 'france', '?']

In [7]:
vocab = {"<UNK>": 0}

In [10]:
def build_vocab(row):
    tokenized_question = tokenize(row["question"])
    tokenized_answer = tokenize(row["answer"])
    merged_tokens = tokenized_question + tokenized_answer

    for token in merged_tokens:
        if token not in vocab:
            vocab[token] = len(vocab)

In [11]:
df.apply(build_vocab, axis = 1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [12]:
vocab

{'<UNK>': 0,
 'what': 1,
 'is': 2,
 'the': 3,
 'capital': 4,
 'of': 5,
 'france': 6,
 '?': 7,
 'paris': 8,
 'germany': 9,
 'berlin': 10,
 'who': 11,
 'wrote': 12,
 "'to": 13,
 'kill': 14,
 'a': 15,
 'mockingbird': 16,
 "'": 17,
 'harper-lee': 18,
 'largest': 19,
 'planet': 20,
 'in': 21,
 'our': 22,
 'solar': 23,
 'system': 24,
 'jupiter': 25,
 'boiling': 26,
 'point': 27,
 'water': 28,
 'celsius': 29,
 '100': 30,
 'painted': 31,
 'mona': 32,
 'lisa': 33,
 'leonardo-da-vinci': 34,
 'square': 35,
 'root': 36,
 '64': 37,
 '8': 38,
 'chemical': 39,
 'symbol': 40,
 'for': 41,
 'gold': 42,
 'au': 43,
 'which': 44,
 'year': 45,
 'did': 46,
 'world': 47,
 'war': 48,
 'ii': 49,
 'end': 50,
 '1945': 51,
 'longest': 52,
 'river': 53,
 'nile': 54,
 'japan': 55,
 'tokyo': 56,
 'developed': 57,
 'theory': 58,
 'relativity': 59,
 'albert-einstein': 60,
 'freezing': 61,
 'fahrenheit': 62,
 '32': 63,
 'known': 64,
 'as': 65,
 'red': 66,
 'mars': 67,
 'author': 68,
 "'1984": 69,
 'george-orwell': 70,
 

In [13]:
len(vocab)

328

In [17]:
def text_to_indices(text, vocab):
    indexed_text = []
    for token in tokenize(text):
        if token in vocab:
            indexed_text.append(vocab[token])
        else:
            indexed_text.append(vocab["<UNK>"])

    return indexed_text

In [21]:
text_to_indices("What is the capital of France?", vocab)

[1, 2, 3, 4, 5, 6, 7]

In [16]:
import torch
from torch.utils.data import Dataset, DataLoader

In [18]:
class QADataset(Dataset):
    def __init__(self, df, vocab):
        self.df = df
        self.vocab = vocab

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        numerical_question = text_to_indices(self.df.iloc[index]["question"], self.vocab)
        numerical_answer = text_to_indices(self.df.iloc[index]["answer"], self.vocab)

        return torch.tensor(numerical_question), torch.tensor(numerical_answer)

In [22]:
dataset = QADataset(df, vocab)

In [23]:
dataloader = DataLoader(dataset, batch_size = 1, shuffle = True)

In [24]:
for question, answer in dataloader:
    print(question, answer)

tensor([[11, 57,  3, 58,  5, 59,  7]]) tensor([[60]])
tensor([[  1,   2,   3,   4,   5, 115,   7]]) tensor([[116]])
tensor([[ 1,  2,  3, 61, 27,  5, 28, 21, 62,  7]]) tensor([[63]])
tensor([[  1,   2,   3, 143, 119,  85,   3, 281, 282,   7]]) tensor([[123]])
tensor([[ 44, 139, 120,   3, 250,   5, 251,   7]]) tensor([[252]])
tensor([[ 11, 312,   3, 313, 314,  17,   7]]) tensor([[315]])
tensor([[ 44, 119, 120,   3, 121,  96, 122,   7]]) tensor([[123]])
tensor([[ 80,  81, 198,  83,  21,   3, 199, 200, 201,   7]]) tensor([[202]])
tensor([[ 1,  2,  3,  4,  5, 55,  7]]) tensor([[56]])
tensor([[ 1,  2,  3, 71,  5, 55,  7]]) tensor([[263]])
tensor([[11, 77, 78,  7]]) tensor([[79]])
tensor([[ 44, 294, 295, 120, 296, 161, 297, 298,   7]]) tensor([[299]])
tensor([[ 80,  81, 292,  83,  21,  15, 293,   7]]) tensor([[87]])
tensor([[  1,   2,   3,  39,  40,  41, 164,   7]]) tensor([[165]])
tensor([[ 44,  88,  89, 244, 245,  21,  41, 246,   7]]) tensor([[247]])
tensor([[ 44, 170,   2,   3,  19, 171, 1

In [26]:
import torch.nn as nn

In [40]:
class RNN(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim = 50)
        self.rnn = nn.RNN(50, 64, batch_first = True)
        self.linear = nn.Linear(64, vocab_size)

    def forward(self, question):
        embeddings_question = self.embedding(question)
        hidden_state, final_state = self.rnn(embeddings_question)
        output = self.linear(final_state).squeeze(0)

        return output

In [45]:
epochs = 100
lr = 0.001
model = RNN(len(vocab))

In [46]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

In [47]:
for epoch in range(epochs):
    total_epochs_loss = 0
    for question, answer in dataloader:
        output = model(question)
        loss = criterion(output, answer[0])
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_epochs_loss += loss.item()
    print(f"Epoch: {epoch + 1}, Loss: {total_epochs_loss}")

Epoch: 1, Loss: 530.9022798538208
Epoch: 2, Loss: 431.1832616329193
Epoch: 3, Loss: 377.97755670547485
Epoch: 4, Loss: 335.35286712646484
Epoch: 5, Loss: 292.5800337791443
Epoch: 6, Loss: 251.60721600055695
Epoch: 7, Loss: 212.40984916687012
Epoch: 8, Loss: 177.58302557468414
Epoch: 9, Loss: 144.49323672056198
Epoch: 10, Loss: 117.03786534070969
Epoch: 11, Loss: 93.68509736657143
Epoch: 12, Loss: 74.6953085064888
Epoch: 13, Loss: 60.30694195628166
Epoch: 14, Loss: 48.63182473182678
Epoch: 15, Loss: 39.29434812068939
Epoch: 16, Loss: 32.46164807677269
Epoch: 17, Loss: 26.731629975140095
Epoch: 18, Loss: 22.40000295639038
Epoch: 19, Loss: 19.003661312162876
Epoch: 20, Loss: 16.302626214921474
Epoch: 21, Loss: 14.092831756919622
Epoch: 22, Loss: 12.378021985292435
Epoch: 23, Loss: 10.856917068362236
Epoch: 24, Loss: 9.672311279922724
Epoch: 25, Loss: 8.57206992059946
Epoch: 26, Loss: 7.709961157292128
Epoch: 27, Loss: 6.964719507843256
Epoch: 28, Loss: 6.323299324139953
Epoch: 29, Loss: 5

In [59]:
def prediction(model, question, threshold = 0.5):
    num_question = text_to_indices(question, vocab)
    tensor_question = torch.tensor(num_question).unsqueeze(0)
    output = model(tensor_question)
    probs = nn.Softmax(dim = 1)(output)
    value, index = torch.max(probs, dim = 1)
    print(value, index)

In [60]:
prediction(model, "What is campusx")

tensor([0.1886], grad_fn=<MaxBackward0>) tensor([34])
